# Controls charts—average and range (XbarR)¶

# In brevi
Shewhart control charts, also called process behaviour charts, are used to determine if the variation of a process is stable and predictable, that is, in a state of statistical control. In-control variation arises from chance or common causes. No changes or adjustments to the process are needed. The chart can be used to predict future performance. Out-of-control variation arises from special or assignable causes. These charts help identify the special causes in order minimize or eliminate their effect.

# Data

Download the [data file](https://drive.google.com/open?id=0BzrdQfHR2I5DdWpLOC1DSWltVzA). It consists of a "Sample" column and several "Xi" columns of floats or integers.

# Methodology
The charts presented here are the average and range control charts, XbarR. The data are collected using rational samples. The average values of each subgroup sample are plotted in time order. A central line (average) and control limits above and below the central line are plotted.

In [1]:
# Import the required libraries and modules.
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
# Read the csv data file. It is encoded in UTF-8.
# The values to plot are in column X.
# Set "date" as the index.
#chart_data = pd.read_csv('xbarr.csv', parse_dates=True, index_col='Date')
chart_data = pd.read_csv('xbarr.csv', index_col='Sample')

In [3]:
# Define for the Xbar chart the graph title and subtitle, and the x and y axis labels.
x_chart_title = 'Xbar Control Chart'
x_chart_subtitle = 'Thickness'
x_chart_ylabel = 'Thickness (mm)'
x_chart_xlabel = 'Sample'
d_two = 2.059
d_three = 0.8798
subgroup_size = 4

In [4]:
# Define for the R chart the graph title and subtitle, and the x and y axis labels.
mr_chart_title = 'R Control Chart'
mr_chart_subtitle = 'Thickness Range'
mr_chart_ylabel = 'Thickness Range (mm)'
mr_chart_xlabel = 'Sample'

In [5]:
# R chart statistics
# Calculate average range
average_range = (chart_data.max(axis='columns') - chart_data.min(axis='columns')).mean()
average_range

0.044833333333333315

In [6]:
# Calculate the range chart upper control limit.
range_upper_control_limit = average_range + 3 * d_three * average_range / d_two
range_upper_control_limit

0.10230448437752951

In [7]:
# Calculate the range chart lower control limit.
range_chart_upper_control_limit = average_range - 3 * d_three * average_range / d_two
range_chart_upper_control_limit

-0.012637817710862871

In [8]:
# Set the moving range lower control limit to 0 if it is < 0.
if range_chart_upper_control_limit < 0:
    range_chart_upper_control_limit = 0
range_chart_upper_control_limit

0

In [9]:
# Xbar chart statistics
# Calculate average of averages.
average_of_averages = (chart_data.mean(axis='columns')).mean()
average_of_averages

0.54256249999999995

In [10]:
# Calculate the averages chart upper control limit.
average_chart_upper_control_limit = average_of_averages + 3 * average_range / \
                              (d_two * (subgroup_size)**0.5)
average_chart_upper_control_limit

0.57522398615832926

In [11]:
# Calculate the averages chart lower control limit.
average_chart_lower_control_limit = average_of_averages - 3 * average_range / \
                              (d_two * (subgroup_size)**0.5)
average_chart_lower_control_limit

0.50990101384167064

# Control chart formulae

## Average chart (X)

$$
    \begin{align}
        UCL_\overline{X}, LCL_\overline{X} & = \overline{\overline{X}} \pm 3 \times \text{Sigma(}\overline{X}\text{)} \\
                     & = \overline{\overline{X}} \pm 3 \times \frac{\text{Sigma(X) }}{\sqrt{n}} \\
                     & = \overline{\overline{X}} \pm \frac{3 \text{ }\overline{R}}{d_2 \sqrt{n}}
    \end{align}
$$

The constant $d_2$ can be found in tables of control chart constants.It is a rescaling constant that changes an average range to a standard deviation. The value of $d_2$ is a function of the subgroup size $n$.

## Range chart (R)

$$
    \begin{align}
        UCL_R, LCL_R & = \overline{R} \pm 3 \times \text{Sigma(R)} \\
                     & = \overline{R} \pm 3 \times d_3 \times \text{Sigma(X)} \\
                     & = \overline{R} \pm 3 \times d_3 \times \frac{\overline{R}}{d_2}
    \end{align}
$$

The constant $d_3$ can be found in tables of control chart constants. It is a rescaling constant that changes a a standard deviation of individual values to a standard deviation of range values. The value of $d_3$ is a function of subgroup size.